# 该文件未完成

In [ ]:
import openmeteo_requests
import requests_cache
from pandas import DataFrame
from retry_requests import retry
import pandas as pd
import matplotlib.pyplot as plt
from PIL import Image


def request_openmeteo_data(start_date, end_date, latitude, longitude, daily):
    # Setup the Open-Meteo API client with cache and retry on error
    cache_session = requests_cache.CachedSession('.cache', expire_after = -1)
    retry_session = retry(cache_session, retries = 5, backoff_factor = 0.2)
    openmeteo = openmeteo_requests.Client(session = retry_session)
    # Make sure all required weather variables are listed here
    # The order of variables in hourly or daily is important to assign them correctly below
    url = "https://archive-api.open-meteo.com/v1/archive"
    params = {
        "start_date": start_date,
        "end_date": end_date,
        "latitude": latitude,
        "longitude": longitude,
        "daily": daily
    }
    responses = openmeteo.weather_api(url, params=params)
    return responses

def process_sample_data(responses):
    df_all_sample = pd.DataFrame()
    for response in responses:
        daily = response.Daily()
        daily_temperature_2m_mean = daily.Variables(0).ValuesAsNumpy()
        daily_precipitation_sum = daily.Variables(1).ValuesAsNumpy()
        daily_soil_moisture_28_to_100cm_mean = daily.Variables(2).ValuesAsNumpy()
        # 日期
        daily_data = {"date": pd.date_range(
            start = pd.to_datetime(daily.Time(), unit = "s", utc = True),
            end = pd.to_datetime(daily.TimeEnd(), unit = "s", utc = True),
            freq = pd.Timedelta(seconds = daily.Interval()),
            inclusive = "left"
        )}
        # 指标
        daily_data["temperature_2m_mean"] = daily_temperature_2m_mean
        daily_data["precipitation_sum"] = daily_precipitation_sum
        daily_data["soil_moisture_28_to_100cm_mean"] = daily_soil_moisture_28_to_100cm_mean
        # 整合
        daily_dataframe = pd.DataFrame(data = daily_data)
        df_all_sample = pd.concat([df_all_sample, daily_dataframe], ignore_index=True)
    # 样本合并
    df_l2 = df_all_sample.groupby('date', as_index=False)[['temperature_2m_mean', 'precipitation_sum', 'soil_moisture_28_to_100cm_mean']].mean()


    return df_l2

In [3]:
import openmeteo_requests

import pandas as pd
import requests_cache
from retry_requests import retry

# Setup the Open-Meteo API client with cache and retry on error
cache_session = requests_cache.CachedSession('.cache', expire_after = -1)
retry_session = retry(cache_session, retries = 5, backoff_factor = 0.2)
openmeteo = openmeteo_requests.Client(session = retry_session)

# Make sure all required weather variables are listed here
# The order of variables in hourly or daily is important to assign them correctly below
url = "https://archive-api.open-meteo.com/v1/archive"
params = {
    "latitude": [0.189201, 1.292902, 1.223743],
    "longitude": [116.900165, 116.93554, 117.829622],
    "start_date": "2025-01-01",
    "end_date": "2025-08-05",
    "daily": ["precipitation_sum", "temperature_2m_mean", "soil_moisture_7_to_28cm_mean"],
}
responses = openmeteo.weather_api(url, params=params)

# Process 3 locations
for response in responses:
    print(f"\nCoordinates: {response.Latitude()}°N {response.Longitude()}°E")
    print(f"Elevation: {response.Elevation()} m asl")
    print(f"Timezone difference to GMT+0: {response.UtcOffsetSeconds()}s")

    # Process daily data. The order of variables needs to be the same as requested.
    daily = response.Daily()
    daily_precipitation_sum = daily.Variables(0).ValuesAsNumpy()
    daily_temperature_2m_mean = daily.Variables(1).ValuesAsNumpy()
    daily_soil_moisture_7_to_28cm_mean = daily.Variables(2).ValuesAsNumpy()

    daily_data = {"date": pd.date_range(
        start = pd.to_datetime(daily.Time(), unit = "s", utc = True),
        end = pd.to_datetime(daily.TimeEnd(), unit = "s", utc = True),
        freq = pd.Timedelta(seconds = daily.Interval()),
        inclusive = "left"
    )}

    daily_data["precipitation_sum"] = daily_precipitation_sum
    daily_data["temperature_2m_mean"] = daily_temperature_2m_mean
    daily_data["soil_moisture_7_to_28cm_mean"] = daily_soil_moisture_7_to_28cm_mean

    daily_dataframe = pd.DataFrame(data = daily_data)
    # print("\nDaily data\n", daily_dataframe)


Coordinates: 0.17574691772460938°N 116.8876724243164°E
Elevation: 13.0 m asl
Timezone difference to GMT+0: 0s

Coordinates: 1.3005272150039673°N 116.94313049316406°E
Elevation: 88.0 m asl
Timezone difference to GMT+0: 0s

Coordinates: 1.2302284240722656°N 117.7742691040039°E
Elevation: 7.0 m asl
Timezone difference to GMT+0: 0s


## open-meteo-api 通用方法

In [12]:
def request_openmeteo_api(latitude:list[str],
                          longitude:list[str],
                          start_date:str,
                          end_date:str,
                          daily:list[str]):
    '''
    The len(latitude) and len(longitude) entered must be equal.

    :param latitude:    [float, ...]
    :param longitude:   [float, ...]
    :param start_date:  "YYYY-MM-DD"
    :param end_date:    "YYYY-MM-DD"
    :param daily:       [String, ...]
    :return:
    '''
    # Setup the Open-Meteo API client with cache and retry on error
    cache_session = requests_cache.CachedSession('.cache', expire_after = -1)
    retry_session = retry(cache_session, retries = 5, backoff_factor = 0.2)
    openmeteo = openmeteo_requests.Client(session = retry_session)

    url = "https://archive-api.open-meteo.com/v1/archive"
    params = {
        "latitude": latitude,
        "longitude": longitude,
        "start_date": start_date,
        "end_date": end_date,
        "daily": daily
    }
    responses = openmeteo.weather_api(url, params=params)


print(request_openmeteo_api.__doc__)





len(latitude) = len(latitude)
:param latitude:    [float, ...]
:param longitude:   [float, ...]
:param start_date:  "YYYY-MM-DD"
:param end_date:    "YYYY-MM-DD"
:param daily:       [String, ...]
:return:

